# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 10 2022 9:38AM,248253,10,7068040,"CLINUVEL, Inc.",Released
1,Oct 10 2022 9:30AM,248252,10,MTR22-0259,"Senseonics, Incorporated",Released
2,Oct 10 2022 9:27AM,248251,12,HH-37376,Hush Hush,Released
3,Oct 10 2022 9:27AM,248251,12,HH-37377,Hush Hush,Released
4,Oct 10 2022 9:27AM,248251,12,HH-37378,Hush Hush,Released
5,Oct 10 2022 9:27AM,248251,12,HH-37379,Hush Hush,Released
6,Oct 10 2022 9:27AM,248251,12,HH-37380,Hush Hush,Released
7,Oct 10 2022 9:27AM,248251,12,HH-37381,Hush Hush,Released
8,Oct 10 2022 9:27AM,248251,12,HH-37382,Hush Hush,Released
9,Oct 10 2022 9:27AM,248251,12,HH-37383,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,248246,Released,3
21,248247,Executing,1
22,248251,Released,29
23,248252,Released,1
24,248253,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Executing,Released
id,,,
248246,NaN,NaN,3.0
248247,NaN,1.0,NaN
248251,NaN,NaN,29.0
248252,NaN,NaN,1.0
248253,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Executing,Released
id,,,
248213,1.0,0.0,22.0
248214,0.0,1.0,0.0
248215,0.0,0.0,1.0
248221,0.0,15.0,13.0
248222,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Cancelled,Executing,Released
id,,,
248213,1.0,0,22
248214,0.0,1,0
248215,0.0,0,1
248221,0.0,15,13
248222,0.0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Executing,Released
0,248213,1.0,0,22
1,248214,0.0,1,0
2,248215,0.0,0,1
3,248221,0.0,15,13
4,248222,0.0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Cancelled,Executing,Released
0,248213,1.0,,22
1,248214,0.0,1,
2,248215,0.0,,1
3,248221,0.0,15,13
4,248222,0.0,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 10 2022 9:38AM,248253,10,"CLINUVEL, Inc."
1,Oct 10 2022 9:30AM,248252,10,"Senseonics, Incorporated"
2,Oct 10 2022 9:27AM,248251,12,Hush Hush
31,Oct 10 2022 9:24AM,248247,12,Hush Hush
32,Oct 10 2022 9:21AM,248245,10,Bio-PRF
71,Oct 10 2022 9:16AM,248246,16,Sartorius Bioprocess Solutions
74,Oct 10 2022 9:11AM,248244,10,"Methapharm, Inc."
77,Oct 10 2022 9:10AM,248241,10,"Methapharm, Inc."
106,Oct 10 2022 9:09AM,248243,12,LF of America Corp.
107,Oct 10 2022 9:09AM,248242,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Executing,Released
0,Oct 10 2022 9:38AM,248253,10,"CLINUVEL, Inc.",0.0,,1
1,Oct 10 2022 9:30AM,248252,10,"Senseonics, Incorporated",0.0,,1
2,Oct 10 2022 9:27AM,248251,12,Hush Hush,0.0,,29
3,Oct 10 2022 9:24AM,248247,12,Hush Hush,0.0,1,
4,Oct 10 2022 9:21AM,248245,10,Bio-PRF,0.0,,39
5,Oct 10 2022 9:16AM,248246,16,Sartorius Bioprocess Solutions,0.0,,3
6,Oct 10 2022 9:11AM,248244,10,"Methapharm, Inc.",0.0,,3
7,Oct 10 2022 9:10AM,248241,10,"Methapharm, Inc.",0.0,,29
8,Oct 10 2022 9:09AM,248243,12,LF of America Corp.,0.0,,1
9,Oct 10 2022 9:09AM,248242,10,"Methapharm, Inc.",0.0,,7


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 10 2022 9:38AM,248253,10,"CLINUVEL, Inc.",1,
1,Oct 10 2022 9:30AM,248252,10,"Senseonics, Incorporated",1,
2,Oct 10 2022 9:27AM,248251,12,Hush Hush,29,
3,Oct 10 2022 9:24AM,248247,12,Hush Hush,,1
4,Oct 10 2022 9:21AM,248245,10,Bio-PRF,39,
5,Oct 10 2022 9:16AM,248246,16,Sartorius Bioprocess Solutions,3,
6,Oct 10 2022 9:11AM,248244,10,"Methapharm, Inc.",3,
7,Oct 10 2022 9:10AM,248241,10,"Methapharm, Inc.",29,
8,Oct 10 2022 9:09AM,248243,12,LF of America Corp.,1,
9,Oct 10 2022 9:09AM,248242,10,"Methapharm, Inc.",7,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 10 2022 9:38AM,248253,10,"CLINUVEL, Inc.",1,
1,Oct 10 2022 9:30AM,248252,10,"Senseonics, Incorporated",1,
2,Oct 10 2022 9:27AM,248251,12,Hush Hush,29,
3,Oct 10 2022 9:24AM,248247,12,Hush Hush,,1
4,Oct 10 2022 9:21AM,248245,10,Bio-PRF,39,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 10 2022 9:38AM,248253,10,"CLINUVEL, Inc.",1.0,NaN
1,Oct 10 2022 9:30AM,248252,10,"Senseonics, Incorporated",1.0,NaN
2,Oct 10 2022 9:27AM,248251,12,Hush Hush,29.0,NaN
3,Oct 10 2022 9:24AM,248247,12,Hush Hush,NaN,1.0
4,Oct 10 2022 9:21AM,248245,10,Bio-PRF,39.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 10 2022 9:38AM,248253,10,"CLINUVEL, Inc.",1.0,0.0
1,Oct 10 2022 9:30AM,248252,10,"Senseonics, Incorporated",1.0,0.0
2,Oct 10 2022 9:27AM,248251,12,Hush Hush,29.0,0.0
3,Oct 10 2022 9:24AM,248247,12,Hush Hush,0.0,1.0
4,Oct 10 2022 9:21AM,248245,10,Bio-PRF,39.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4468202,264.0,1.0
12,744741,30.0,1.0
16,248246,3.0,0.0
19,248221,13.0,15.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4468202,264.0,1.0
1,12,744741,30.0,1.0
2,16,248246,3.0,0.0
3,19,248221,13.0,15.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,264.0,1.0
1,12,30.0,1.0
2,16,3.0,0.0
3,19,13.0,15.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,264.0
1,12,Released,30.0
2,16,Released,3.0
3,19,Released,13.0
4,10,Executing,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19
Status,,,,
Executing,1.0,1.0,0.0,15.0
Released,264.0,30.0,3.0,13.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19
0,Executing,1.0,1.0,0.0,15.0
1,Released,264.0,30.0,3.0,13.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19
0,Executing,1.0,1.0,0.0,15.0
1,Released,264.0,30.0,3.0,13.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()